In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window

spark = SparkSession.builder.getOrCreate()
spark

## Window rangeBetween

In [53]:
df = spark.createDataFrame(
    [
        (10, "a"),
        (20, "a"),
        (30, "a"),
        (40, "b"),
        (50, "b"),
        (60, "b"),
    ],
    ["id", "category"],
)
df.show()


+---+--------+
| id|category|
+---+--------+
| 10|       a|
| 20|       a|
| 30|       a|
| 40|       b|
| 50|       b|
| 60|       b|
+---+--------+



In [52]:
window = (
    Window
        # 先根据 category 进行分区, 构成一个个的窗
        .partitionBy("category")
        # 在每个窗内, 根据 id 进行排序
        .orderBy("id")
        # 为每一行建立一个小型滑窗, 这个语句的意思是对于每行 (我们称之为主要行) 而言, 根据 orderBy 的排序键 id 有一个值, 
        # 然后向后扫描所有满足 id 位于主要行的 id 值以及主要行 id 值加上 offset (这里是 25) 之间的行作为滑窗
        # 在本例中对于 id10 而言, 所有的 id 在 10 <= x <= 35 之间的行都会被包括进来, 也就是 id10, id20, id30 三行
        # 而对于 id 20 而言, 所有的 id 在 20 <= x <= 45 之间的行都会被包括进来, 也就是 id20 和 id30 两行
        .rangeBetween(Window.currentRow, 25)
)
(
    df.withColumn(
        "sum", 
        # 对于每个小滑窗, 我们对 id 进行求和
        F.sum("id").over(window),
    )
    .sort("id", "category")
    .show()
)


# 以下计算结果是这样得来的:
#
# (10, a, 60) 中的 sum = 60 来自于 id10 + id20 + id30
# (20, a, 50) 中的 sum = 50 来自于 id20 + id30
# (30, a, 30) 中的 sum = 30 来自于 id30
# (40, b, 150) 中的 sum = 150 来自于 id40 + id50 + id60
# (50, b, 110) 中的 sum = 110 来自于 id50 + id60
# (60, b, 60) 中的 sum = 60 来自于 id60


+---+--------+---+
| id|category|sum|
+---+--------+---+
|  1|       a| 10|
|  3|       a|  9|
|  6|       a|  6|
| 10|       b| 25|
| 15|       b| 15|
| 21|       b| 21|
+---+--------+---+



## Window rowsBetween

``pyspark.sql.Window.rowsBetween`` 是用来为在一个窗口中的每一行建立一个小滑窗, 然后对滑窗内的值进行计算. 举例来说, 我们的数据表有一个 partition key ``category``, 每一个 ``category`` 就是一个 Window. 然后对于 Window 中的每一行, 我们取该行以及它的下一行这两行构成一个小型滑窗 (如果这一行已经是最后一行了, 那么就没有下一行, 滑窗内也就只有一行). 下面我们来看一个例子:

In [29]:
df = spark.createDataFrame(
    [
        (10, "a"),
        (20, "a"),
        (30, "a"),
        (40, "b"),
        (50, "b"),
        (60, "b"),
    ],
    ["id", "category"],
)
df.show()

+---+--------+
| id|category|
+---+--------+
| 10|       a|
| 20|       a|
| 30|       a|
| 40|       b|
| 50|       b|
| 60|       b|
+---+--------+



In [31]:
window = (
    Window
        # 先根据 category 进行分区, 构成一个个的窗
        .partitionBy("category")
        # 在每个窗内, 根据 id 进行排序
        .orderBy("id")
        # 为每一行建立一个小型滑窗, 这个小型滑窗由该行以及它的下一行 (offset = 1) 构成
        .rowsBetween(Window.currentRow, 1)
)

(

    df.withColumn(
        "sum", 
        # 对于每个小滑窗, 我们对 id 进行求和
        F.sum("id").over(window),
    )
    .sort("id", "category", "sum")
    .show()
)

# 以下计算结果是这样得来的:
#
# (10, a, 30) 中的 sum = 3 来自于 id10 + id20
# (20, a, 50) 中的 sum = 5 来自于 id20 + id30
# (30, a, 30) 中的 sum = 3 来自于 id30 + id40 (不存在)
# (40, b, 90) 中的 sum = 9 来自于 id40 + id50
# (50, b, 110) 中的 sum = 11 来自于 id50 + id60
# (60, b, 60) 中的 sum = 6 来自于 id60 + id70 (不存在)


+---+--------+---+
| id|category|sum|
+---+--------+---+
| 10|       a| 30|
| 20|       a| 50|
| 30|       a| 30|
| 40|       b| 90|
| 50|       b|110|
| 60|       b| 60|
+---+--------+---+

